In [1]:
import os
import json
import requests
import pandas as pd
import time


In [2]:
google_place = os.environ["GOOGLE_API"]

In [3]:
# SkyTrain station information is retrieved from the csv file
SkyTrainDF = pd.read_csv('skytrain_stations.csv')

## Google Places API Request
Due to limitations placed by Google, 3 pages of 20 results each will be retrieved for each SkyTrain station. The total number of restaurants retrieved from Google Places ends up at an even 1920.

Some information that was collected from Yelp could not be collected from Google Places in the same way. URL, Category, and distance from the station will remain absent from the data collected via Google Places.

In [58]:
# Define API key and endpoint
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'

# Initialize an empty list to store SkyTrain station details
station_details = []

# Iterate over each station to retrieve coordinates and name
for station in SkyTrainDF.iterrows():
    station_latitude = str(
        station[1]['Latitude']
    )
    station_longitude = str(
        station[1]['Longitude']
    )
    station_name = str(
        station[1]['Name']
    )
    # Define the parameters for request
    params = {
        'query': 'Restaurants in Vancouver',
        'radius': 500,  # Radius in meters
        'location': { 'lat': station_latitude, 'lng': station_longitude},
        'key': google_place
    }

    # Send the initial request
    response = requests.get(url, params=params)

    # Parse the response as JSON
    data = response.json()


    # Handle the response data
    if response.status_code == 200:
        results = data['results']

        for result in results:
            name = result['name']
            address = result['formatted_address']
            rating = result.get('rating', 'N/A')
            review_count = result['user_ratings_total']
            place_id = result['place_id']
            location = result['geometry']['location']
            latitude = location['lat']
            longitude = location['lng']
            skytrain_station = station_name

            station_details.append({
                'Venue Name': name,
                'Address': address,
                'Average Rating': rating,
                'Review Count': review_count,
                'SkyTrain Station': station_name,
                'Latitude': latitude,
                'Longitude': longitude,
                'ID': place_id,
                'Address': address,
                'Source': 'Google'
            })

        # Check if there are more results and retrieve them using pagination
        while 'next_page_token' in data:
            next_page_token = data['next_page_token']
            params['pagetoken'] = next_page_token

            # allow the next page token to become valid
            time.sleep(3)

            # Send the next page request
            response = requests.get(url, params=params)

            # Parse the response as JSON
            data = response.json()

            if response.status_code == 200:
                results = data['results']

                for result in results:
                    name = result['name']
                    address = result['formatted_address']
                    rating = result.get('rating', 'N/A')
                    review_count = result['user_ratings_total']
                    place_id = result['place_id']
                    location = result['geometry']['location']
                    latitude = location['lat']
                    longitude = location['lng']
                    skytrain_station = station_name
                        
                    station_details.append({
                        'Venue Name': name,
                        'Address': address,
                        'Average Rating': rating,
                        'Review Count': review_count,
                        'SkyTrain Station': station_name,
                        'Latitude': latitude,
                        'Longitude': longitude,
                        'ID': place_id,
                        'Address': address,
                        'Source': 'Google'
                    })
            else:
                print('Request failed with status code:', response.status_code)
    else:
        print('Request failed with status code:', response.status_code)

# Create a DataFrame from the station details list
google_rest = pd.DataFrame(station_details)

# the DataFrame
#google_rest

In [63]:
google_rest.drop_duplicates(inplace=True)
google_rest.to_csv('google_venues.csv', index=False)

In [60]:
google_rest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1920 entries, 0 to 1919
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Venue Name        1920 non-null   object 
 1   Address           1920 non-null   object 
 2   Average Rating    1920 non-null   float64
 3   Review Count      1920 non-null   int64  
 4   SkyTrain Station  1920 non-null   object 
 5   Latitude          1920 non-null   float64
 6   Longitude         1920 non-null   float64
 7   ID                1920 non-null   object 
 8   Source            1920 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 150.0+ KB


In [65]:
google_rplace = pd.read_csv('google_venues.csv')
google_rplace

,Venue Name,Address,Average Rating,Review Count,SkyTrain Station,Latitude,Longitude,ID,Source
0,Chambar Restaurant,"568 Beatty St, Vancouver, BC V6B 2L3, Canada",4.5,3529,Broadway-City Hall Station,49.280127,-123.109935,ChIJq2tI6HtxhlQRBaKUv59cS9E,Google
1,Cardero's Restaurant,"1583 Coal Harbour Quay, Vancouver, BC V6G 3E7,...",4.5,5137,Broadway-City Hall Station,49.291610,-123.127482,ChIJOQXpDoZxhlQRwQ2A6Ulh9-w,Google
2,Elisa,"1109 Hamilton St, Vancouver, BC V6B 5P6, Canada",4.5,1836,Broadway-City Hall Station,49.275964,-123.121413,ChIJhcHC8TBzhlQRPsVWKk8ZNJQ,Google
3,Giardino Restaurant,"1328 Hornby St, Vancouver, BC V6Z 1W5, Canada",4.6,781,Broadway-City Hall Station,49.276941,-123.129993,ChIJdd9ipdNzhlQRTeCkanpMc8s,Google
4,Nightingale,"1017 W Hastings St, Vancouver, BC V6E 0C4, Canada",4.5,2490,Broadway-City Hall Station,49.287398,-123.117475,ChIJD9jfUoJxhlQR2Z9sjekAEFg,Google
...,...,...,...,...,...,...,...,...,...
1915,Jules Bar à Vin,"216 Abbott St, Vancouver, BC V6B 2K8, Canada",4.4,691,Columbia,49.283435,-123.106387,ChIJi9Ix4HlxhlQRR9d6rVYjsFA,Google
1916,Water St. Café,"300 Water St, Vancouver, BC V6B 1B6, Canada",4.2,1455,Columbia,49.284111,-123.109114,ChIJ28IuM3hxhlQRJGgeE-4_z58,Google
1917,Showcase Restaurant & Bar,"1122 W Hastings St, Vancouver, BC V6E 4J6, Canada",4.2,317,Columbia,49.287858,-123.120336,ChIJUwY964NxhlQRslxEl4a2VZM,Google
1918,Gringo Gastown,"27 Blood Alley Square, Vancouver, BC V6B 0C4, ...",4.5,1825,Columbia,49.283200,-123.105361,ChIJFWhLV3dxhlQR-BCOL1u24FY,Google
